In [1]:
import pandas as pd

In [2]:
train_path = '../Data/widsdatathon2024-university/train.csv'
test_path = '../Data/widsdatathon2024-university/test.csv'

climate_paths = ['../Data/climate_data/training.csv',
                '../Data/climate_data/test.csv']

export_path = '../Data/train_test_added_climate_data.csv'

In [3]:
train_df = pd.read_csv(train_path)
train_df['allocated_set'] = 'train'
test_df = pd.read_csv(test_path)
test_df['allocated_set'] = 'test'

df = pd.concat([train_df,test_df],
              axis=0).reset_index(drop=True)

In [4]:
df.describe()

,patient_id,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_year,population,density,age_median,age_under_10,age_10_to_19,...,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,treatment_pd
count,39519.000000,39519.000000,39519.000000,12482.000000,39519.000000,39519.000000,39519.000000,39519.000000,39519.000000,39519.000000,...,39519.000000,39519.000000,39519.000000,39518.000000,39514.000000,39514.000000,39518.000000,39518.000000,39519.000000,27674.000000
mean,550399.374174,520.365116,54.424049,29.229523,2016.486424,19887.925833,1656.668336,40.689346,11.118332,12.886369,...,5.346427,6.452429,17.788920,13.437071,13.318639,4.614742,28.100588,8.552149,7.072006,134.218472
std,260773.401143,292.656985,11.217219,5.662184,1.057922,13489.353665,3642.158636,4.012954,1.532715,1.881145,...,5.956250,3.518786,17.493325,3.798747,5.760003,6.687422,4.918609,4.317622,3.102473,187.759654
min,100051.000000,6.000000,18.000000,14.000000,2015.000000,11.000000,0.821739,20.600000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.600000,3.433333,0.000000,12.460784,1.165385,1.200000,0.000000
25%,323049.000000,292.000000,47.000000,24.960000,2016.000000,8917.000000,167.167647,37.734426,10.149254,11.792453,...,1.307353,3.892593,4.608824,10.270492,9.329577,0.972414,24.986420,5.472464,4.956250,36.000000
50%,552167.000000,488.000000,55.000000,28.660000,2016.000000,18259.000000,627.904167,40.768605,11.022034,12.923944,...,3.355556,5.578689,11.588889,13.085185,12.122857,2.605882,27.828125,7.382500,6.882222,70.000000
75%,776035.000000,782.000000,62.000000,33.000000,2017.000000,27537.000000,1555.107692,43.212963,12.194737,13.930435,...,7.630000,8.611905,24.485185,15.657143,16.408333,5.958333,30.925000,10.676316,8.640000,133.000000
max,999996.000000,999.000000,91.000000,97.000000,2018.000000,71374.000000,29851.685710,73.500000,17.675000,35.300000,...,33.188889,26.429870,99.104348,35.155556,48.931818,74.067391,48.020000,29.565000,65.700000,1455.000000


In [5]:
df.columns

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'breast_cancer_diagnosis_year', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_treatment', 'metastatic_first_treatment_type',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'region', 'division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_5

# split breast cancer diagnosis into sides and quadrants

In [6]:
map_cancer_desc = {
    'Malignant neoplasm of ovrlp sites of right female breast': {'side':'R', 'quadrant':'overlap'},
    'Malig neoplasm of upper-outer quadrant of left female breast': {'side':'L', 'quadrant':'UO'},
    'Malig neoplasm of upper-inner quadrant of left female breast': {'side':'L', 'quadrant':'UI'},
    'Malignant neoplasm of breast (female), unspecified': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplm of lower-inner quadrant of right female breast': {'side':'R', 'quadrant':'LI'},
    'Malignant neoplasm of upper-inner quadrant of female breast': {'side':'unspecified', 'quadrant':'UI'},
    'Malignant neoplasm of central portion of left female breast': {'side':'L', 'quadrant':'central'},
    'Malignant neoplasm of unspecified site of left female breast': {'side':'L', 'quadrant':'unspecified'},
    'Malig neoplm of upper-inner quadrant of right female breast': {'side':'R', 'quadrant':'UI'},
    'Malignant neoplasm of unsp site of unspecified female breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of unsp site of right female breast': {'side':'R', 'quadrant':'unspecified'},
    'Malig neoplm of upper-outer quadrant of right female breast': {'side':'R', 'quadrant':'UO'},
    'Malig neoplasm of lower-inner quadrant of left female breast': {'side':'L', 'quadrant':'LI'},
    'Malignant neoplasm of nipple and areola, left female breast': {'side':'L', 'quadrant':'nipple'},
    'Malignant neoplasm of lower-outer quadrant of female breast': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of other specified sites of female breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplm of lower-outer quadrant of right female breast': {'side':'R', 'quadrant':'LO'},
    'Malignant neoplasm of central portion of breast, female': {'side':'unspecified', 'quadrant':'central'},
    'Malignant neoplasm of lower-inner quadrant of female breast': {'side':'unspecified', 'quadrant':'LI'},
    'Malignant neoplasm of ovrlp sites of left female breast': {'side':'L', 'quadrant':'overlap'},
    'Malignant neoplasm of central portion of right female breast': {'side':'R', 'quadrant':'central'},
    'Malignant neoplasm of nipple and areola, right female breast': {'side':'R', 'quadrant':'nipple'},
    'Malignant neoplasm of axillary tail of left female breast': {'side':'L', 'quadrant':'axillary tail'},
    'Malignant neoplasm of upper-outer quadrant of female breast': {'side':'unspecified', 'quadrant':'UO'},
    'Malig neoplasm of upper-outer quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'UO'},
    'Malig neoplasm of lower-outer quadrant of left female breast': {'side':'L', 'quadrant':'LO'},
    'Malignant neoplasm of central portion of unsp female breast': {'side':'unspecified', 'quadrant':'central'},
    'Malig neoplasm of lower-outer quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of axillary tail of right female breast': {'side':'R', 'quadrant':'axillary tail'},
    'Malignant neoplasm of ovrlp sites of unsp female breast': {'side':'unspecified', 'quadrant':'overlap'},
    'Malignant neoplasm of upper-outer quadrant of breast, female': {'side':'unspecified', 'quadrant':'UO'},
    'Malignant neoplasm of overlapping sites of breast, female': {'side':'unspecified', 'quadrant':'overlap'},
    'Malignant neoplasm of central portion of female breast': {'side':'unspecified', 'quadrant':'central'},
    'Malignant neoplasm of axillary tail of breast, female': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of breast of unspecified site, female': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplasm of upper-inner quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'UI'},
    'Secondary malignant neoplasm of breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of lower-outer quadrant of breast, female': {'side':'unspecified', 'quadrant':'LO'},
    'Malignant neoplasm of axillary tail of unsp female breast': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of nipple and areola, unsp female breast': {'side':'unspecified', 'quadrant':'nipple'},
    'Malig neoplasm of lower-inner quadrant of unsp female breast': {'side':'unspecified', 'quadrant':'LI'},
    'Malignant neoplasm of axillary tail of female breast': {'side':'unspecified', 'quadrant':'axillary tail'},
    'Malignant neoplasm of unsp site of unspecified male breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malig neoplasm of lower-outer quadrant of right male breast': {'side':'R', 'quadrant':'LO'},
    'Malig neoplasm of upper-inner quadrant of left male breast': {'side':'L', 'quadrant':'UI'},
    'Malignant neoplasm of upper-inner quadrant of breast, female': {'side':'unspecified', 'quadrant':'UI'},
    'Malignant neoplasm of nipple and areola of female breast': {'side':'unspecified', 'quadrant':'nipple'},
    'Malignant neoplasm of nipple and areola, female': {'side':'unspecified', 'quadrant':'nipple'},
    'Malignant neoplasm of nipple and areola, right male breast': {'side':'R', 'quadrant':'nipple'},
    'Malignant neoplasm of central portion of right male breast': {'side':'R', 'quadrant':'central'},
    'Malignant neoplasm of lower-inner quadrant of breast, female': {'side':'unspecified', 'quadrant':'LI'},
    'Malig neoplasm of lower-outer quadrant of left male breast': {'side':'L', 'quadrant':'LO'},
    'Malig neoplasm of upper-outer quadrant of right male breast': {'side':'R', 'quadrant':'UO'},
    'Malig neoplasm of upper-inner quadrant of right male breast': {'side':'R', 'quadrant':'UI'},
    'Malignant neoplasm of other and unspecified sites of male breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of unspecified site of right male breast': {'side':'R', 'quadrant':'unspecified'},
    'Malig neoplasm of lower-inner quadrant of left male breast': {'side':'L', 'quadrant':'LI'},
    'Malig neoplasm of upper-outer quadrant of left male breast': {'side':'L', 'quadrant':'UO'},
    'Malignant neoplasm of breast': {'side':'unspecified', 'quadrant':'unspecified'},
    'Malignant neoplasm of nipple and areola, left male breast': {'side':'L', 'quadrant':'nipple'},
    'Malignant neoplasm of axillary tail of right male breast': {'side':'R', 'quadrant':'axillary tail'},
}

In [7]:
df['side'] = df['breast_cancer_diagnosis_desc'].apply(lambda x: map_cancer_desc[x]['side'])
df['quadrant'] = df['breast_cancer_diagnosis_desc'].apply(lambda x: map_cancer_desc[x]['quadrant'])

In [8]:
df['side'].value_counts(dropna=False)

side
L              15031
R              14783
unspecified     9705
Name: count, dtype: int64

In [9]:
df['quadrant'].value_counts(dropna=False)

quadrant
unspecified      19146
UO                8419
overlap           3787
UI                2499
central           1741
LO                1700
LI                1125
nipple             781
axillary tail      321
Name: count, dtype: int64

# group metastasis codes into organs

In [10]:
ICD10_code_description_dict = {
    'C770':"Secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck", 
 'C771': "Secondary and unspecified malignant neoplasm of intrathoracic lymph nodes",
 'C772': "Secondary and unspecified malignant neoplasm of intra-abdominal lymph nodes", 
 'C773': "Secondary and unspecified malignant neoplasm of axilla and upper limb lymph nodes",
 'C774': "Secondary and unspecified malignant neoplasm of inguinal and lower limb lymph nodes",
 'C775': "Secondary and unspecified malignant neoplasm of intrapelvic lymph nodes",
 'C778': "Secondary and unspecified malignant neoplasm of lymph nodes of multiple regions",
 'C779': "Secondary and unspecified malignant neoplasm of lymph node, unspecified",
 'C7800': "Secondary malignant neoplasm of unspecified lung",
 'C7801': "Secondary malignant neoplasm of right lung",
 'C7802': "Secondary malignant neoplasm of left lung",
 'C781': "Secondary malignant neoplasm of mediastinum",
 'C782': "Secondary malignant neoplasm of pleura",
 'C7830': "Secondary malignant neoplasm of unspecified respiratory organ",
 'C7839': "Secondary malignant neoplasm of other respiratory organs",
 'C784': "Secondary malignant neoplasm of small intestine" ,
 'C785':"Secondary malignant neoplasm of large intestine and rectum",
'C786':"Secondary malignant neoplasm of retroperitoneum and peritoneum",
'C787': "Secondary malignant neoplasm of liver and intrahepatic bile duct",
 'C7880': "Secondary malignant neoplasm of unspecified digestive organ",
'C7889': "Secondary malignant neoplasm of other digestive organs",
 'C7900': "Secondary malignant neoplasm of unspecified kidney and renal pelvis", 
 'C7901': "Secondary malignant neoplasm of right kidney and renal pelvis", 
 'C7902': "Secondary malignant neoplasm of left kidney and renal pelvis",
'C7910': "Secondary malignant neoplasm of unspecified urinary organs",
'C792': "Secondary malignant neoplasm of skin",
 'C7931': "Secondary malignant neoplasm of brain", 
 'C7932': "Secondary malignant neoplasm of cerebral meninges",
 'C7940': "Secondary malignant neoplasm of unspecified part of nervous system", 
 'C7949': "Secondary malignant neoplasm of other parts of nervous system",
'C7951': "Secondary malignant neoplasm of bone",
'C7952': "Secondary malignant neoplasm of bone marrow" ,
 'C7960': "Secondary malignant neoplasm of unspecified ovary",
'C7961': "Secondary malignant neoplasm of right ovary",
'C7962': "Secondary malignant neoplasm of left ovary",
 'C7970':"Secondary malignant neoplasm of unspecified adrenal gland",
 'C7971': "Secondary malignant neoplasm of right adrenal gland",
'C7972': "Secondary malignant neoplasm of left adrenal gland",
'C7981':"Secondary malignant neoplasm of breast",
'C7982': "Secondary malignant neoplasm of genital organs",
'C7989': "Secondary malignant neoplasm of other specified sites",
 'C799': "Secondary malignant neoplasm of unspecified site"}

In [11]:
map_metas_code_to_organ = {
    'C770':"lymph nodes of head, face and neck", 
 'C771': "intrathoracic lymph nodes",
 'C772': "intra-abdominal lymph nodes", 
 'C773': "axilla and upper limb lymph nodes",
 'C774': "inguinal and lower limb lymph nodes",
 'C775': "intrapelvic lymph nodes",
 'C778': "lymph nodes of multiple regions",
 'C779': "lymph node, unspecified",
 'C7800': "lung",
 'C7801': "lung",
 'C7802': "lung",
 'C781': "mediastinum",
 'C782': "lung",
 'C7830': "lung",
 'C7839': "lung",
 'C784': "intestine" ,
 'C785':"intestine",
'C786':"retroperitoneum and peritoneum",
'C787': "liver",
 'C7880': "unspecified digestive organ",
'C7889': "unspecified digestive organ",
 'C7900': "kidney and other urinary organs", 
 'C7901': "kidney and other urinary organs", 
 'C7902': "kidney and other urinary organs",
'C7910': "kidney and other urinary organs",
    'C7911': 'kidney and other urinary organs',
'C792': "skin",
 'C7931': "brain", 
 'C7932': "brain",
 'C7940': "brain", 
 'C7949': "brain",
'C7951': "bone",
'C7952': "bone" ,
 'C7960': "ovary",
'C7961': "ovary",
'C7962': "ovary",
 'C7970':"adrenal gland",
 'C7971': "adrenal gland",
'C7972': "adrenal gland",
'C7981':"unspecified",
'C7982': "ovary",
'C7989': "unspecified",
 'C799': "unspecified"}

In [12]:
df['metastatic_organ'] = df['metastatic_cancer_diagnosis_code'].apply(lambda x: map_metas_code_to_organ[x])

In [13]:
df['metastatic_organ'].value_counts(dropna=False)

metastatic_organ
axilla and upper limb lymph nodes      26298
bone                                    3967
unspecified                             2411
lymph node, unspecified                 1736
lung                                    1467
liver                                   1382
brain                                    458
intrathoracic lymph nodes                455
lymph nodes of head, face and neck       409
skin                                     317
lymph nodes of multiple regions          249
intra-abdominal lymph nodes               99
retroperitoneum and peritoneum            84
mediastinum                               62
unspecified digestive organ               26
adrenal gland                             25
ovary                                     23
intestine                                 23
kidney and other urinary organs           12
inguinal and lower limb lymph nodes       10
intrapelvic lymph nodes                    6
Name: count, dtype: int64

In [14]:
map_treatment_to_generic = {
     'ATEZOLIZUMAB':'ATEZOLIZUMAB',
     'BEVACIZUMAB':'BEVACIZUMAB',
     'BLEOMYCIN SULFATE':'BLEOMYCIN',
     'CAPECITABINE':'CAPECITABINE',
     'CARBOPLATIN':'CARBOPLATIN',
     'CISPLATIN':'CISPLATIN',
     'CYCLOPHOSPHAMIDE':'CYCLOPHOSPHAMIDE',
     'DOCETAXEL':'DOCETAXEL',
     'DOCETAXEL ANHYDROUS':'DOCETAXEL',
     'DOXORUBICIN HCL':'DOXORUBICIN',
     'DOXORUBICIN HCL LIPOSOMAL':'DOXORUBICIN',
     'DOXORUBICIN HYDROCHLORIDE':'DOXORUBICIN',
     'DRUG ASSAY EVEROLIMUS':'EVEROLIMUS',
     'EPIRUBICIN HCL':'EPIRUBICIN',
     'EPIRUBICIN HYDROCHLORIDE':'EPIRUBICIN',
     'ERIBULIN MESYLATE':'ERIBULIN',
     'ERLOTINIB HYDROCHLORIDE':'ERLOTINIB',
     'ETOPOSIDE':'ETOPOSIDE',
     'EVEROLIMUS':'EVEROLIMUS',
     'GEMCITABINE HCL':'GEMCITABINE',
     'GEMCITABINE HYDROCHLORIDE':'GEMCITABINE',
     'IPILIMUMAB':'IPILIMUMAB',
     'IRINOTECAN HCL':'IRINOTECAN',
     'IXABEPILONE':'IXABEPILONE',
     'Inj gemcitabine hcl (accord)':'GEMCITABINE',
     'METHOTREXATE':'METHOTREXATE',
     'METHOTREXATE SODIUM':'METHOTREXATE',
     'NIRAPARIB':'NIRAPARIB',
     'NIVOLUMAB':'NIVOLUMAB',
     'PACLITAXEL':'PACLITAXEL',
     'PACLITAXEL PROTEIN BOUND PARTICLES':'PACLITAXEL',
     'PAZOPANIB HYDROCHLORIDE':'PAZOPANIB',
     'PEMETREXED DISODIUM':'PEMETREXED',
     'PEMETREXED DISODIUM HEPTAHYDRATE':'PEMETREXED',
     'RUCAPARIB CAMSYLATE':'RUCAPARIB',
     'TALAZOPARIB TOSYLATE':'TALAZOPARIB',
     'TEMOZOLOMIDE':'TEMOZOLOMIDE',
     'THALIDOMIDE':'THALIDOMIDE',
     'TOPOTECAN HCL':'TOPOTECAN',
     'VINORELBINE TARTRATE':'VINORELBINE'
}

In [15]:
df['cleaned_metastatic_first_treatment'] = df['metastatic_first_treatment'].apply(lambda x:map_treatment_to_generic[x])

In [16]:
df['cleaned_metastatic_first_treatment'].value_counts(dropna=False)

cleaned_metastatic_first_treatment
DOXORUBICIN         14703
CARBOPLATIN          6490
CYCLOPHOSPHAMIDE     5952
PACLITAXEL           4982
CAPECITABINE         2368
DOCETAXEL            1849
EVEROLIMUS            769
ERIBULIN              555
GEMCITABINE           544
METHOTREXATE          348
BEVACIZUMAB           316
VINORELBINE           310
CISPLATIN             150
IXABEPILONE            70
NIVOLUMAB              24
IRINOTECAN             19
TEMOZOLOMIDE           15
IPILIMUMAB              9
PEMETREXED              9
ERLOTINIB               8
EPIRUBICIN              5
ATEZOLIZUMAB            5
ETOPOSIDE               4
TOPOTECAN               4
PAZOPANIB               3
BLEOMYCIN               3
TALAZOPARIB             2
THALIDOMIDE             1
RUCAPARIB               1
NIRAPARIB               1
Name: count, dtype: int64

In [17]:
map_generic_to_type = {
 'ATEZOLIZUMAB':'monoclonal antibody',
 'BEVACIZUMAB':'monoclonal antibody',
 'BLEOMYCIN':'chemotherapy',
 'CAPECITABINE':'chemotherapy',
 'CARBOPLATIN':'chemotherapy',
 'CISPLATIN':'chemotherapy',
 'CYCLOPHOSPHAMIDE':'chemotherapy',
 'DOCETAXEL':'chemotherapy',
 'DOXORUBICIN':'chemotherapy',
 'EPIRUBICIN':'chemotherapy',
 'ERIBULIN':'chemotherapy',
 'ERLOTINIB':'small molecule inhibitor',
 'ETOPOSIDE':'chemotherapy',
 'EVEROLIMUS':'small molecule inhibitor',
 'GEMCITABINE':'chemotherapy',
 'IPILIMUMAB':'monoclonal antibody',
 'IRINOTECAN':'chemotherapy',
 'IXABEPILONE':'chemotherapy',
 'METHOTREXATE':'chemotherapy',
 'NIRAPARIB':'small molecule inhibitor',
 'NIVOLUMAB':'monoclonal antibody',
 'PACLITAXEL':'chemotherapy',
 'PAZOPANIB':'small molecule inhibitor',
 'PEMETREXED':'chemotherapy',
 'RUCAPARIB':'small molecule inhibitor',
 'TALAZOPARIB':'small molecule inhibitor',
 'TEMOZOLOMIDE':'chemotherapy',
 'THALIDOMIDE':'chemotherapy',
 'TOPOTECAN':'chemotherapy',
 'VINORELBINE':'chemotherapy'
}

In [18]:
df['cleaned_metastatic_first_treatment_type'] = df['cleaned_metastatic_first_treatment'].apply(lambda x:map_generic_to_type[x])


In [19]:
df['cleaned_metastatic_first_treatment_type'].value_counts(dropna=False)

cleaned_metastatic_first_treatment_type
chemotherapy                38381
small molecule inhibitor      784
monoclonal antibody           354
Name: count, dtype: int64

In [20]:
df['metastatic_first_novel_treatment'].value_counts(dropna=False)

metastatic_first_novel_treatment
NaN              39190
PEMBROLIZUMAB      174
OLAPARIB           155
Name: count, dtype: int64

# add climate data

In [21]:
climate_df = pd.concat(
    [pd.read_csv(path) for path in climate_paths],
    axis=0,
    ignore_index=True
)

In [22]:
print(df.shape)
print(climate_df.shape)

(39519, 89)
(18698, 83)


In [23]:
climate_df = climate_df.drop_duplicates(subset=['patient_zip3','Ozone','PM25','N02'],
                                       ignore_index=True)
print(climate_df.shape)

(762, 83)


In [24]:
len(df['patient_zip3'].unique())

860

In [25]:
len(climate_df['patient_zip3'].unique())

762

In [26]:
len(climate_df['Ozone'].unique())

757

In [27]:
len(climate_df['PM25'].unique())

757

In [28]:
len(climate_df['N02'].unique())

757

In [29]:
df = df.merge(climate_df[['patient_zip3','Ozone','PM25','N02']],
             how='left',
             on=['patient_zip3'])
print(df.shape)

(39519, 92)


In [30]:
df

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,treatment_pd,allocated_set,side,quadrant,metastatic_organ,cleaned_metastatic_first_treatment,cleaned_metastatic_first_treatment_type,Ozone,PM25,N02
0,994155,Asian,COMMERCIAL,CA,917,46,F,27.00,C50811,Malignant neoplasm of ovrlp sites of right fem...,...,35.0,train,R,overlap,"lymph node, unspecified",DOXORUBICIN,chemotherapy,47.310325,9.595719,20.084231
1,154389,NaN,MEDICARE ADVANTAGE,OH,451,63,F,NaN,C50412,Malig neoplasm of upper-outer quadrant of left...,...,33.0,train,L,UO,bone,DOXORUBICIN,chemotherapy,40.544389,8.399666,12.485563
2,387343,NaN,COMMERCIAL,TX,773,53,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,24.0,train,L,UI,axilla and upper limb lymph nodes,PACLITAXEL,chemotherapy,38.696786,9.361774,11.522807
3,921275,Hispanic,MEDICAID,CA,928,50,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,455.0,train,unspecified,unspecified,liver,GEMCITABINE,chemotherapy,42.301121,8.487175,20.113179
4,803454,NaN,COMMERCIAL,NY,112,39,F,18.00,1749,"Malignant neoplasm of breast (female), unspeci...",...,162.0,train,unspecified,unspecified,unspecified,DOXORUBICIN,chemotherapy,37.722740,7.879795,27.496367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39514,430644,White,MEDICARE ADVANTAGE,OH,443,70,F,33.00,C50911,Malignant neoplasm of unsp site of right femal...,...,NaN,test,R,unspecified,axilla and upper limb lymph nodes,DOXORUBICIN,chemotherapy,38.128975,8.811391,16.755649
39515,695045,NaN,COMMERCIAL,MD,212,55,F,35.87,C50412,Malig neoplasm of upper-outer quadrant of left...,...,NaN,test,L,UO,axilla and upper limb lymph nodes,CARBOPLATIN,chemotherapy,40.668109,8.019797,25.540078
39516,417489,Black,MEDICAID,SC,296,44,F,NaN,C50911,Malignant neoplasm of unsp site of right femal...,...,NaN,test,R,unspecified,lung,DOXORUBICIN,chemotherapy,42.050404,7.893646,11.280694
39517,623596,Hispanic,COMMERCIAL,CA,947,70,F,NaN,C50412,Malig neoplasm of upper-outer quadrant of left...,...,NaN,test,L,UO,axilla and upper limb lymph nodes,CARBOPLATIN,chemotherapy,30.939316,6.243063,18.201716


In [31]:
df.columns

Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'breast_cancer_diagnosis_year', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_treatment', 'metastatic_first_treatment_type',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'region', 'division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_5

In [159]:
# write file

In [160]:
df.to_csv(export_path, index=False)